** The code runs bit slow, please give it some time

Question 1 Pt 1

In [37]:
import urllib.request
from bs4 import BeautifulSoup

In [38]:
seed_url = "https://press.un.org/en"
urls = [seed_url] 
seen = []    
opened = []  
withcrisis = []
counter = 0

In [39]:
while len(urls) > 0:
    try:
        curr_url=urls.pop(0)
        req = urllib.request.Request(curr_url,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        opened.append(curr_url)

    except Exception as ex:
        continue  

    soup = BeautifulSoup(webpage, features="html.parser") 
    text = soup.get_text().lower()
    flag = bool(soup.find('a', {'href': '/en/press-release', 'hreflang': 'en'}))
    if "crisis" in text and flag : 
        counter += 1
        with open(f"1_{counter}.txt", 'w', encoding='utf-8') as f:
            f.write(soup.prettify())
            
        if counter >= 10:
            break

            
    for tag in soup.find_all('a', href = True):
        childUrl = tag['href'] 
        o_childurl = childUrl
        childUrl = urllib.parse.urljoin(seed_url, childUrl)
        if seed_url in childUrl and childUrl not in seen:
            urls.append(childUrl)
            seen.append(childUrl)

/Users/zoe/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Question 1 Pt 2

In [41]:
import urllib.request
from bs4 import BeautifulSoup

In [42]:
seed_url = "https://www.europarl.europa.eu/news/en/press-room/page/{}"
urls = []
page = 0

In [43]:
while len(urls) < 10:
    curr_url = seed_url.format(page)
    req = urllib.request.Request(curr_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(webpage, features="html.parser")

    for tag in soup.find_all('h3', class_="ep-a_heading ep-layout_level2"):
        plenary_session_tag = tag.find('span', class_="ep_name", string='Plenary session')
        text_tag = tag.find('span', class_="ep_name")

        if text_tag:
            text = text_tag.get_text().strip()

            if 'crisis' in text.lower() and plenary_session_tag:
                link = tag.find('a', href=True)
                if link:
                    childUrl = urllib.parse.urljoin(seed_url, link['href'])
                    
                    child_req = urllib.request.Request(childUrl, headers={'User-Agent': 'Mozilla/5.0'})
                    child_webpage = urllib.request.urlopen(child_req).read()

                    filename = f"2_{len(urls)+1}.txt"
                    with open(filename, 'w', encoding='utf-8') as file:
                        file.write(child_webpage.decode('utf-8'))
                    urls.append([text, childUrl])

    page += 1
